# Prefix-tuning sentiment classification 예제 - sentences_allagree 데이터셋
## Reference : https://huggingface.co/docs/peft/task_guides/seq2seq-prefix-tuning

# 필요한 라이브러리 설치

In [ ]:
!pip install -q peft transformers datasets

# 설정값 지정

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = "cuda"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

# sentences_allagree 데이터셋 불러오기
## (financial_phrasebank 데이터셋의 부분집합 데이터셋으로 **금융 뉴스의 감정(sentiment)를 분류**하는 데이터셋입니다.)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Russian export duties will activate harvesting in Finland , and sales in Russia will increase also .',
 'label': 2,
 'text_label': 'positive'}

# Tokenizer 불러오기 & 데이터셋 전처리

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels

    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

# PEFT 모델 설정

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# Prefix-Tuning 기법으로 인해 전체 모델의 0.13%의 파라미터만 Fine-Tuning에 사용

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# Training 시작

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")


100%|██████████| 29/29 [00:06<00:00,  4.66it/s]


epoch=0: train_ppl=tensor(4.5480, device='cuda:0') train_epoch_loss=tensor(1.5147, device='cuda:0') eval_ppl=tensor(1.1151, device='cuda:0') eval_epoch_loss=tensor(0.1089, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.37it/s]


epoch=1: train_ppl=tensor(1.1441, device='cuda:0') train_epoch_loss=tensor(0.1346, device='cuda:0') eval_ppl=tensor(1.0925, device='cuda:0') eval_epoch_loss=tensor(0.0885, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.38it/s]


epoch=2: train_ppl=tensor(1.1004, device='cuda:0') train_epoch_loss=tensor(0.0956, device='cuda:0') eval_ppl=tensor(1.0546, device='cuda:0') eval_epoch_loss=tensor(0.0532, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.36it/s]


epoch=3: train_ppl=tensor(1.0677, device='cuda:0') train_epoch_loss=tensor(0.0655, device='cuda:0') eval_ppl=tensor(1.0466, device='cuda:0') eval_epoch_loss=tensor(0.0456, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.34it/s]

epoch=4: train_ppl=tensor(1.0776, device='cuda:0') train_epoch_loss=tensor(0.0748, device='cuda:0') eval_ppl=tensor(1.0448, device='cuda:0') eval_epoch_loss=tensor(0.0438, device='cuda:0')


# 학습이 끝난 모델 성능 평가

In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=96.47577092511013 % on the evaluation dataset
eval_preds[:10]=['negative', 'neutral', 'negative', 'positive', 'neutral', 'negative', 'neutral', 'positive', 'positive', 'neutral']
dataset['validation']['text_label'][:10]=['negative', 'neutral', 'negative', 'positive', 'neutral', 'negative', 'neutral', 'positive', 'positive', 'neutral']


# 학습이 끝난 모델을 sample text에 대한 Inference

In [ ]:
inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

In [ ]:
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
